# Réalisation d'une analyse statistique avec le langage R
Dans un premier temps, nous allons récupérer les données dans le CSV

In [ ]:
# Charger les données
source("charger.R")
mondata <- charger(2105482)
Y <- mondata$Sales
X1 <- mondata$Price
X2 <- mondata$Advertising
X3 <- mondata$Region

In [ ]:
# Afficher les données
mondata

# Histogramme des ventes

In [ ]:
hist(Y, col="gold",border="red", main=paste("Histogramme des ventes"),
     xlab="Ventes",ylab="Fréquences")

# Diagramme de Tukey pour les ventes

In [ ]:
boxplot(Y,
horizontal=TRUE,
col="gold",border="red",
xlab="Nombre de vente")

# Droite de Henry

In [ ]:
qqnorm(Y, col ="gold", main=paste("Droite de Henry"))
qqline(Y, col ="red")


# Test de normalité (Shapiro-Wilk)

In [ ]:
shapiro.test(Y)

# Interprétation du test de normalité (Shapiro-Wilk)
La statistique W représente la correlation entre les quantiles de notre variable de vente et les quantiles théoriques d'une normale de moyenne 0 et de variance 1.
Dans notre cas w = 0.99463 ce qui est très prêt de 1 donc les quantiles de notre variable sont très prêt de ce d'une normale.
De plus notre p-value = 0.7128 est très élevée comparativement à 0.05
Nous pouvons donc dire que nos données sont conformes à une distribution normale  

# Tableau de statistique descriptive

In [ ]:
mesureStatistique = data.frame(mondata=c("Sales"), 
                     moyenne=NA, écartType=NA, q1=NA, mediane=NA,
                     q3=NA, IntervalConfiance=NA)
# moyenne,
mesureStatistique$moyenne = sapply(2, function(i) mean(Y)) 
# quartiles q1, q2 = mediane, q3 
mesureStatistique[1, c("q1", "mediane", "q3")] = quantile(Y, probs = c(0.25,0.5,0.75))
# écart type,
mesureStatistique$écartType = sapply(2, function(i) sd(Y)) 
# intervalle de confiance pour la moyenne
ci <- t.test(Y, conf.level = 0.95)$conf.int
# cat("95% Confidence Interval: [", ci[1], ", ", ci[2], "]\n")
mesureStatistique$IntervalConfiance = sprintf("[%f, %f]", ci[1], ci[2])

# Afficher le resultat
mesureStatistique

# PARTIE C: Meilleur modèle

## Étude du modèle 1
Y = beta0 + beta1*X1 + epsilon

### Graphe de regression

In [ ]:
modele1 <- lm(Y ~ X1)

# Create a scatter plot of log(Y) against X1
plot(X1, Y, main = "Regression linéaire: Ventes ~ Prix", col = "gold",
     xlab = "Prix", ylab = "Ventes", pch = 16)

# Add the regression line
abline(modele1, col = "red")

### Tableau d'analyse de la variance

In [ ]:
summary(modele1)
anova(modele1)

#### Test de la signification du modèle
Les hypothèses sont les suivantes :
-	H0 : β1 = 0 ce qui implique qu’il n’y a pas de relation linéaire entre Y et X1
-	H1 : β1 ≠ 0
La p-value de β1 est 3.3e-08 qui est inférieure au seuil α = 0 ce qui signifie qu’on rejette l’hypothèse H0
Analyse des résidus
Observons les différents les différents tests effectués sur les résidus avec la figure 1.1.2 ci-dessous.


### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele1)

#### Interprétations:
-	Pour le graphe des residual vs Fitted et le graphe Residual vs Leverage on remarque on remarque une répartition des données assez égale de part et d’autre de l’axe 0. On conclut donc qu’il y a homoscédasticité
-	Pour le graphique Normal Q-Q on constate que les résidus sont majoritairement confondus aux quantiles théoriques de la normal on conclut donc que la distribution de l’erreur suit effectivement une normal
-	En observant le graphe Scale-Location, on remarque que les résidus sont aléatoirement dispersés et ne présentent aucune tendance entre eux. On peut donc conclure qu’il n’y a pas d’autocorrélation entre les résidus 
Ces interprétations montrent que les hypothèses de bases sont respectées par notre modèle, cependant nous avons obtenu comme valeur de R2 :   R2 = 0.1466 qui est très éloigné de 1 nous concluons donc que ce modèle ne convient pas pour faire une prédiction.


### Interval de confiance de beta0 et beta1

In [ ]:
conf_int <- confint(modele1)
conf_int

## Étude du modèle 4
Y = beta0 + beta1*X2 + epsilon

### Graphe de regression

In [ ]:
modele4 <- lm(Y ~ X2)

# Create a scatter plot of log(Y) against X1
plot(X2, Y, main = "Regression linéaire: Ventes ~ Prix", col = "gold",
     xlab = "Prix", ylab = "Ventes", pch = 16)

# Add the regression line
abline(modele4, col = "red")

### Tableau d'analyse de la variance

In [ ]:
summary(modele4)
anova(modele4)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele4)

### Interval de confiance de beta0 et beta1

In [ ]:
conf_int <- confint(modele4)
conf_int

## Étude du modèle 2
Y = beta0*(X1^beta1)*exp(epsilon)

### Graphe de regression

In [ ]:
modele2 <- lm(log(Y) ~ log(X1))

# Create a scatter plot of log(Y) against X1
plot(log(X1), log(Y), main = "Regression linéaire: log(Ventes) ~ log(Prix)", col = "gold",
     xlab = "log(Prix)", ylab = "log(Ventes)", pch = 16)

# Add the regression line
abline(modele2, col = "red")

### Tableau d'analyse de la variance

In [ ]:
summary(modele2)
anova(modele2)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele2)

### Interval de confiance de beta0 et beta1

In [ ]:
# Obtenir les intervalles de confiance des coefficients
conf_int <- confint(modele2) # Calculer les intervalles de confiance
# Afficher les résultats
conf_int


## Étude du modèle 5
Y = beta0*(X2^beta1)*exp(epsilon)

### Graphe de regression

In [ ]:
modele5 <- lm(log(Y) ~ log(X2+8))

# Create a scatter plot of log(Y) against X2
plot(log(X2+1), log(Y), main = "Regression linéaire: log(Ventes) ~ log(publicité)", col = "gold",
     xlab = "log(Publicites)", ylab = "log(Vente)", pch = 16)

# Add the regression line
abline(modele5, col = "red")

### Tableau d'analyse de la variance

In [ ]:
summary(modele5)
anova(modele5)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele5)

### Interval de confiance de beta0 et beta1

In [ ]:
# Obtenir les intervalles de confiance des coefficients
conf_int <- confint(modele5) # Calculer les intervalles de confiance
# Afficher les résultats
conf_int


## Étude du modèle 3
Y = beta0 * exp[(beta1 * X1) + epsilon]

### Graphe de regression

In [ ]:
modele3 <- lm(log(Y) ~ (X1))

# Create a scatter plot of log(Y) against X1
plot(X1, log(Y), main = "Regression linéaire: log(Ventes) ~ prix", col = "gold",
     xlab = "Prix", ylab = "log(Vente)", pch = 16)

# Add the regression line
abline(modele3, col = "red")

### Tableau d'analyse de la variance

In [ ]:
summary(modele3)
anova(modele3)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele3)

### Interval de confiance de beta0 et beta1

In [ ]:
# Obtenir les intervalles de confiance des coefficients
conf_int <- confint(modele3) # Calculer les intervalles de confiance
# Afficher les résultats
conf_int


## Étude du modèle 6
Y = beta0 * exp[(beta1 * X2) + epsilon]

### Graphe de regression

In [ ]:
modele6 <- lm(log(Y) ~ X2)

# Create a scatter plot of log(Y) against X1
plot(X2, log(Y), main = "Regression linéaire: log(Ventes) ~ publicité", col = "gold",
     xlab = "Publicites", ylab = "log(Vente)", pch = 16)

# Add the regression line
abline(modele6, col = "red")

### Tableau d'analyse de la variance

In [ ]:
summary(modele6)
anova(modele6)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele6)

### Interval de confiance de beta0 et beta1

In [ ]:
# Obtenir les intervalles de confiance des coefficients
conf_int <- confint(modele6) # Calculer les intervalles de confiance
# Afficher les résultats
conf_int


## Choix du meilleur modèle:

## Prévision des ventes en fonction du meilleur modele

In [ ]:
predict